In [1]:
# importing necessary modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, learning_curve
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.feature_selection import SelectKBest, f_regression

In [2]:
# import data

df = pd.read_csv("listings_eda")
df.head()

,name,borough,neighbourhood,room_type_encoded,price,bathrooms,accommodates,bedrooms,price_range_encoded,minimum_nights,availability_365
0,"Spacious Brooklyn Duplex, Patio + Garden",Brooklyn,Sunset Park,0,275,1.0,4,2.0,2,21,267
1,Cozy Clean Guest Room - Family Apt,Manhattan,Upper West Side,1,75,1.0,1,1.0,0,2,0
2,BlissArtsSpace!,Brooklyn,Bedford-Stuyvesant,1,60,1.0,2,1.0,1,30,322
3,Large Furnished Room Near B'way,Manhattan,Midtown,1,68,1.0,2,1.0,1,2,79
4,Uptown Sanctuary w/ Private Bath (Month to Month),Manhattan,East Harlem,1,65,1.0,1,1.0,1,30,300


In [3]:
len(df.columns)
df.isna().sum()

name                   0
borough                0
neighbourhood          0
room_type_encoded      0
price                  0
bathrooms              0
accommodates           0
bedrooms               0
price_range_encoded    0
minimum_nights         0
availability_365       0
dtype: int64

In [4]:
len(df.neighbourhood.unique()) -1 + len(df.borough.unique()) - 1 + 11

236

In [5]:
#creating the dummies and merging them back to the original dataframe
cols_to_encode = ['neighbourhood', 'borough']
dummies = pd.get_dummies(df[cols_to_encode], prefix=cols_to_encode, drop_first=True)
df = pd.concat([df, dummies], axis=1)


In [6]:
df.head()

,name,borough,neighbourhood,room_type_encoded,price,bathrooms,accommodates,bedrooms,price_range_encoded,minimum_nights,...,neighbourhood_Willowbrook,neighbourhood_Windsor Terrace,neighbourhood_Woodhaven,neighbourhood_Woodlawn,neighbourhood_Woodrow,neighbourhood_Woodside,borough_Brooklyn,borough_Manhattan,borough_Queens,borough_Staten Island
0,"Spacious Brooklyn Duplex, Patio + Garden",Brooklyn,Sunset Park,0,275,1.0,4,2.0,2,21,...,0,0,0,0,0,0,1,0,0,0
1,Cozy Clean Guest Room - Family Apt,Manhattan,Upper West Side,1,75,1.0,1,1.0,0,2,...,0,0,0,0,0,0,0,1,0,0
2,BlissArtsSpace!,Brooklyn,Bedford-Stuyvesant,1,60,1.0,2,1.0,1,30,...,0,0,0,0,0,0,1,0,0,0
3,Large Furnished Room Near B'way,Manhattan,Midtown,1,68,1.0,2,1.0,1,2,...,0,0,0,0,0,0,0,1,0,0
4,Uptown Sanctuary w/ Private Bath (Month to Month),Manhattan,East Harlem,1,65,1.0,1,1.0,1,30,...,0,0,0,0,0,0,0,1,0,0


In [7]:
df.price.mean()

178.00104113831122

In [8]:
len(df) * .7, len(df) *.3

(26221.3, 11237.699999999999)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns='price'),
                                                    df.price, test_size=0.3,
                                                    random_state=18)

In [10]:
X_train.shape, X_test.shape

((26221, 235), (11238, 235))

In [11]:
names_list = ['name', 'borough','neighbourhood']
names_train = X_train[names_list]
names_test = X_test[names_list]
X_train.drop(columns=names_list, inplace=True)
X_test.drop(columns=names_list, inplace=True)
X_train.shape, X_test.shape

((26221, 232), (11238, 232))

In [12]:
X_train.dtypes.value_counts(), X_test.dtypes.value_counts()

(uint8      225
 int64        5
 float64      2
 dtype: int64,
 uint8      225
 int64        5
 float64      2
 dtype: int64)

I'll start off by using the mean as a predictor to establish a baseline.

In [13]:
dumb_reg = DummyRegressor(strategy='mean')
dumb_reg.fit(X_train, y_train)
dumb_reg.constant_

array([[177.9889783]])

In [14]:
y_tr_pred = dumb_reg.predict(X_train)
y_te_pred = dumb_reg.predict(X_test)

In [15]:
r2_score(y_train, y_tr_pred), r2_score(y_test, y_te_pred)

(0.0, -5.927664825655654e-08)

In [16]:
mse_tr = mean_squared_error(y_train, y_tr_pred)
mse_te = mean_squared_error(y_test, y_te_pred)
print(mse_tr, mse_te)

27075.6863511967 27274.05522294035


In [17]:
rmse = np.sqrt(mse_te)
print("By using the average price of the listings as a predictor, we would be off by an average of ${} from the true values".format(round(rmse,2)))

By using the average price of the listings as a predictor, we would be off by an average of $165.15 from the true values


And now a randomforest to compare

In [18]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=18)
rfr.fit(X_train, y_train)
y_pred = rfr.predict(X_test)

In [19]:
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
print("R-squared:", r2)
print("MSE:", mse)

R-squared: 0.8558859698915364
MSE: 3930.5737825874853


In [20]:
rmse = np.sqrt(mse)
print("This random forest model's predictions would be off by an average of ${} from the true values".format(round(rmse,2)))

This random forest model's predictions would be off by an average of $62.69 from the true values


In the next sectioon I will tune the parameters of the RF model and cross validate its performance. 

In [21]:
df.to_csv('modelling_data', index=False)